## "Research to optimize Hyperparameters of LightGBM with Customized Sharp weighted"

# Apply GPU to LightGBM
- [GPU-accelerated LightGBM full](https://www.kaggle.com/dromosys/gpu-accelerated-lightgbm-full)

In [1]:
%%time
!rm -r /opt/conda/lib/python3.6/site-packages/lightgbm
!git clone --recursive https://github.com/Microsoft/LightGBM

rm: cannot remove '/opt/conda/lib/python3.6/site-packages/lightgbm': No such file or directory
Cloning into 'LightGBM'...
remote: Enumerating objects: 20724, done.
remote: Total 20724 (delta 0), reused 0 (delta 0), pack-reused 20724
Receiving objects: 100% (20724/20724), 16.22 MiB | 24.32 MiB/s, done.
Resolving deltas: 100% (15127/15127), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/kaggle/working/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728

In [2]:
%%time
!apt-get install -y -qq libboost-all-dev

CPU times: user 29.1 ms, sys: 13.1 ms, total: 42.2 ms
Wall time: 2.43 s


In [3]:
%%time
%%bash
cd LightGBM
rm -r build
mkdir build
cd build
cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
make -j$(nproc)

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_2_2
-- Looking for CL_VERSION_2_2 - not found
-- Looking for CL_VERSION_2_1
-- Looking for CL_VERSION_2_1 - not found
-- Looking for CL_VERSION_2_0
-- Looking for CL_VERSION_2_0 - not found
-- Looking for CL_VERSION_1_2
-- Looking

rm: cannot remove 'build': No such file or directory
cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/u

CPU times: user 41.4 ms, sys: 32.1 ms, total: 73.5 ms
Wall time: 5min 8s


In [4]:
%%time
!cd LightGBM/python-package/;python3 setup.py install --precompile

running install
running build
running build_py
creating build
creating build/lib
creating build/lib/lightgbm
copying lightgbm/plotting.py -> build/lib/lightgbm
copying lightgbm/basic.py -> build/lib/lightgbm
copying lightgbm/engine.py -> build/lib/lightgbm
copying lightgbm/callback.py -> build/lib/lightgbm
copying lightgbm/libpath.py -> build/lib/lightgbm
copying lightgbm/compat.py -> build/lib/lightgbm
copying lightgbm/sklearn.py -> build/lib/lightgbm
copying lightgbm/dask.py -> build/lib/lightgbm
copying lightgbm/__init__.py -> build/lib/lightgbm
running egg_info
creating lightgbm.egg-info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'build'
writing manifest file 'lightgbm.e

In [5]:
%%time
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
!rm -r LightGBM

CPU times: user 8.13 ms, sys: 8.18 ms, total: 16.3 ms
Wall time: 1.67 s


# Required library

In [6]:
%%time
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os
import random
import datatable as dt
import seaborn as sns
from tqdm.notebook import tqdm
import plotly.graph_objects as go
from sklearn.model_selection import TimeSeriesSplit, KFold, train_test_split
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args
import re
import pickle
from tqdm.notebook import tqdm
import lightgbm as lgb
import optuna
from optuna.samplers import TPESampler
import multiprocessing
import numba
import datatable as dtable
from functools import partial
from sklearn import preprocessing
import gc
from sklearn.metrics import roc_auc_score
import cudf

CPU times: user 1.48 s, sys: 525 ms, total: 2.01 s
Wall time: 4.84 s


# Initial conditions

In [7]:
%%time
SEED = 20
num_boost_round = int(20)
early_stopping_rounds = int(20)
n_trials = int(1)

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 10.5 µs


In [8]:
%%time
# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(SEED)

CPU times: user 132 µs, sys: 0 ns, total: 132 µs
Wall time: 154 µs


# Preprocess

In [9]:
%%time
train_cudf  = cudf.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train = train_cudf.to_pandas()
del train_cudf
print ("Data is loaded!")

Data is loaded!
CPU times: user 9.4 s, sys: 6.22 s, total: 15.6 s
Wall time: 1min 21s


In [10]:
%%time
train = train[train['weight'] != 0]
train['action'] = (train['resp'] > 0).astype('int')
X_train = train.loc[:, train.columns.str.contains('feature')]
y_train = train.loc[:, 'action']
groups = train['date'].values
weightes = train['weight'].values
resps = train['resp'].values
del train

CPU times: user 890 ms, sys: 1.68 s, total: 2.57 s
Wall time: 2.51 s


# Functions

In [11]:
%%time
def group_time_split(groups: np.ndarray, splits, X):
    group_list = np.unique(groups)
    n_samples = len(X)
    n_groups = len(group_list)
    indices = np.arange(n_samples)
    test_size = n_groups // splits
    test_starts = range(test_size + n_groups % splits, n_groups, test_size)
    test_starts = list(test_starts)
    for test_start in test_starts:
        yield (
            indices[np.isin(groups, group_list[:test_start])],
            indices[np.isin(groups, group_list[test_start:])],
        )


def utility_score_last(date, weight, resp, action):
    count_i = date[-1] + 1
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u


def get_opt_threshold(pred, y_true, groups, weights, bins=50):
    p_min, p_max = pred.min(), pred.max()
    opt_u = 0
    opt_th = p_min
    for th in np.linspace(p_min, p_max, endpoint=False, num=bins):
        act = np.where(pred > th, 1, 0)
        u = utility_score_last(groups, weights, y_true, act)
        if u > opt_u:
            opt_u = u
            print(u)
            opt_th = th
    return opt_th

def optimize(params, X_train: pd.DataFrame, y_train: pd.DataFrame, groups: np.ndarray, splits=5):
    print(params)
    weighted_auc = 0
    div = splits * (splits - 1) / 2

    for i, (tr_idx, val_idx) in enumerate(group_time_split(groups, splits, X_train)):
        tr_X = X_train.iloc[tr_idx]
        tr_y = y_train.iloc[tr_idx]
        val_X = X_train.iloc[val_idx]
        val_y = y_train.iloc[val_idx]
        
        clf = lgb.LGBMClassifier(
            task = 'train',
            boosting_type = 'gbdt',
            objective = 'binary',
            random_state = SEED,
            importance_type = 'gain',
            n_estimators = num_boost_round,
            n_jobs = multiprocessing.cpu_count(),
            metric = 'auc'
        )
        clf.set_params(**params)
        clf.fit(
            tr_X, 
            tr_y, 
            eval_set=[(val_X, val_y)], 
            eval_metric='auc',
            early_stopping_rounds=early_stopping_rounds,
            verbose=False
        )
        val_pred = clf.predict_proba(val_X)[:, 1]

        weighted_auc += (i + 1) * roc_auc_score(val_y, val_pred) / div

        gc.collect()

    return - weighted_auc

CPU times: user 6 µs, sys: 3 µs, total: 9 µs
Wall time: 12.9 µs


# Optimisation

In [12]:
%%time
def objective(trial):
    params = {
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.1, 0.9),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.1, 0.9),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0
    }
    return optimize(params, X_train, y_train, groups, splits=5)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.63 µs


In [13]:
%%time
# Optimize Hyperparameters.
study = optuna.create_study(direction = 'maximize', sampler = TPESampler(seed=int(SEED)))
study.optimize(objective, n_trials = n_trials, n_jobs = multiprocessing.cpu_count())

[I 2020-12-31 12:27:56,894] A new study created in memory with name: no-name-87130736-80fc-48e7-8437-cb8ceb5f6b88


{'lambda_l1': 0.7755443155940888, 'lambda_l2': 0.43260585448507877, 'num_leaves': 198, 'feature_fraction': 0.6304512045259768, 'bagging_fraction': 0.6878990226261725, 'bagging_freq': 3, 'min_child_samples': 23, 'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0}


[I 2020-12-31 12:30:46,375] Trial 0 finished with value: -0.5218277457478044 and parameters: {'lambda_l1': 0.7755443155940888, 'lambda_l2': 0.43260585448507877, 'num_leaves': 198, 'feature_fraction': 0.6304512045259768, 'bagging_fraction': 0.6878990226261725, 'bagging_freq': 3, 'min_child_samples': 23}. Best is trial 0 with value: -0.5218277457478044.


CPU times: user 4min 29s, sys: 14 s, total: 4min 43s
Wall time: 2min 49s


# Baseline Classifier Model

In [14]:
%%time
Class = lgb.LGBMClassifier()
Class.set_params(**study.best_trial.params)
Class.fit(
    X_train,
    y_train
)

[LightGBM] [Warning] lambda_l1 is set=0.7755443155940888, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7755443155940888
[LightGBM] [Warning] feature_fraction is set=0.6304512045259768, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6304512045259768
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l2 is set=0.43260585448507877, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.43260585448507877
[LightGBM] [Warning] bagging_fraction is set=0.6878990226261725, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6878990226261725
CPU times: user 4min 1s, sys: 2.34 s, total: 4min 3s
Wall time: 2min 12s


LGBMClassifier(bagging_fraction=0.6878990226261725, bagging_freq=3,
               feature_fraction=0.6304512045259768,
               lambda_l1=0.7755443155940888, lambda_l2=0.43260585448507877,
               min_child_samples=23, num_leaves=198)

# Predict & Submit

In [15]:
%%time
weighted_opt = 0.5

import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

for (test_df, sample_prediction_df) in iter_test:
    if test_df['weight'].item() > 0:
        X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
        y_preds = Class.predict_proba(X_test)
        y_preds = np.where(y_preds>weighted_opt, 1, 0)
        sample_prediction_df.action = y_preds
    else:
        sample_prediction_df.action = 0
    env.predict(sample_prediction_df)

CPU times: user 7min 31s, sys: 9.43 s, total: 7min 41s
Wall time: 4min 10s


# Save files

In [16]:
%%time
pickle.dump(study.best_trial.params, open('the_best_hyperparameter.pickle', 'wb'))
pickle.dump(X_train, open('X_train.pickle', 'wb'))
pickle.dump(y_train, open('y_train.pickle', 'wb'))
pickle.dump(groups, open('groups.pickle', 'wb'))
pickle.dump(weightes, open('weightes.pickle', 'wb'))
pickle.dump(resps, open('resps.pickle', 'wb'))

CPU times: user 673 ms, sys: 3.03 s, total: 3.71 s
Wall time: 5.22 s


# References
1. Notebooks
    - [Expanding-window & Sharp Weighted Return](https://www.kaggle.com/haozhuai/expanding-window-sharp-weighted-return)
    - [xgb_classifier + validation + utility scoring func](https://www.kaggle.com/rpygamer/xgb-classifier-validation-utility-scoring-func)
    - [Optimized LightGBM Classifier on Utility Score](https://www.kaggle.com/satorushibata/optimized-lightgbm-classifier-on-utility-score)
    - [Market Prediction: XGBoost with GPU (Fit in 1min)](https://www.kaggle.com/hamditarek/market-prediction-xgboost-with-gpu-fit-in-1min)
    - [GPU-accelerated LightGBM full](https://www.kaggle.com/dromosys/gpu-accelerated-lightgbm-full)
1. Discussion
    - [Customized Sharp weighted return for Lightgbm & Xgboost](https://www.kaggle.com/c/jane-street-market-prediction/discussion/200915)
    - [Kaggle kernel LightGBM gpu](https://www.kaggle.com/c/avito-demand-prediction/discussion/56852)

# Acknowledgment
### [@WangMH](https://www.kaggle.com/haozhuai), I'm really thankful to you !
### And your Notebook: [Expanding-window & Sharp Weighted Return](https://www.kaggle.com/haozhuai/expanding-window-sharp-weighted-return) is a great work for me.